In [ ]:
import pyodbc
import pandas as pd
from time import gmtime, strftime
from alpha_vantage.timeseries import TimeSeries
import time
import QuantLib as ql
from utilities.date_utilities import *


In [ ]:
connection = pyodbc.connect('Driver=SQL Server;'
                      'Server=DESKTOP-0CQQR1E\SQLEXPRESS;'
                      'Database=securities_master;'
                      'Trusted_Connection=yes;')
cursor=connection.cursor()
ts = TimeSeries(key='NHXLC1G323WY6E6I', output_format='pandas')

In [ ]:

def check_update(ticker):
    """
    check if a  ticker's history is None or the latest update is performed more (excluding) than 1 data before

    :param ticker:
    :return: -999 for not None, float for business days since last update
    """
    global connection
    proceed_update=False
    
    

    
    data = pd.read_sql_query(
        f"select top 1 *  from securities_hist where ticker='{ticker}' order by eff_date desc", connection)

    if len(data)<1: # first-time update
        return -999
    else:
        current_time = pd.to_datetime(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
        last_updated_time = data["eff_date"][0]
        business_day_since_update = business_day_between(last_updated_time,current_time)
        return business_day_since_update,data["eff_date"][0]
    





In [ ]:
check_update("A")

In [ ]:


def insert_sec_full(ticker,sec_id,sleep_time=14):
    global connection,cursor,ts
    b_days_since_last_eff, last_eff = check_update(ticker) 

    if b_days_since_last_eff==-999: # security is new, loading all data
        print(f"{ticker} does not exist in the database, fetching full history...")
        data = ts.get_daily_adjusted(ticker, outputsize='full')[0]
  
    elif b_days_since_last_eff>=2: # security exists but not up to date
        print(f"{ticker} is {b_days_since_last_eff} days behind, fetching new data...")
        data = ts.get_daily_adjusted(ticker, outputsize='full')[0]
        data = data[:last_eff].iloc[:-1]
        time.sleep(sleep_time)
    else:
        print(f"{ticker} is up to date, no action needed.")
        return 
    time.sleep(sleep_time)
    
    if len(data)>0:
        
        
        for idx in range(len(data)):
            record=data.iloc[idx]
            curr_time=strftime("%Y-%m-%d %H:%M:%S", gmtime())
            insert_str = f"""
            INSERT INTO securities_hist
            (
            sec_id,
            ticker,
            instrument,
            eff_date,
            open_price,
            high_price,
            low_price,
            close_price,
            adj_close_price,
            volume,
            dividend,
            split,
            created_date,
            last_updated_date)
            values (

            '{sec_id}',
            '{ticker}',
            '{'Common Stock'}',
            '{record.name}',
            {record['1. open']},
            {record['2. high']},
            {record['3. low']},
            {record['4. close']},
            {record['5. adjusted close']},
            {record['6. volume']},
            {record['7. dividend amount']},
            {record['8. split coefficient']},
            '{curr_time}',
            '{curr_time}'

            )
            """

            cursor.execute(insert_str)
            connection.commit()
            print(f"{ticker} updated.")







In [ ]:


static_sec_info = pd.read_sql_query("select * from securities", connection)


# update tickers

for  ticker,sec_id in zip(sorted(static_sec_info.ticker.values)[:1],sorted(static_sec_info.sec_id.values)):
    try:

        insert_sec_full(ticker,sec_id)
    except Exception:
        print(f"fatal error {ticker}")
        time.sleep(30)
        continue

        


connection.close()



In [ ]:
connection.close()